In [15]:
import pandas as pd
from datetime import timedelta

In [16]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
accounts = pd.read_csv("../Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

## 4) Devise a method of generating a table of all Accounts with their “nominal” expected payments as of any arbitrary timestamp in the past. In other words, we’re looking for the amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.

In [ ]:
# groups.head()

In [17]:
# Create new column with the number of payments needed to pay the 'price_unlock' amount if the minimum payment was made each time.
groups['max_num_payments'] = (groups['price_unlock'] - groups['price_upfront']) / groups['minimum_payment']
# groups.head()

## Merge Dataframes

In [18]:
# Merge 'ast' and 'accounts' dataframes
merged_df = pd.merge(left=ast, right=accounts, how='outer', left_on='account_id', right_on='id')

# Limit columns
merged_df = merged_df[['started_when','account_id','to_state','group_id','is_unlocked']]
merged_df.head()

,started_when,account_id,to_state,group_id,is_unlocked
0,2019-11-06 05:49:39.571392,2,ENABLED,1,True
1,2019-11-25 22:20:59.150339,2,DISABLED,1,True
2,2019-11-26 10:22:36.571392,2,ENABLED,1,True
3,2019-11-29 11:03:32.150339,2,DISABLED,1,True
4,2019-12-03 10:08:03.571392,2,ENABLED,1,True


In [19]:
# Merge with 'groups' dataframe
merged_df = pd.merge(left=merged_df, right=groups, how='outer', left_on='group_id', right_on='id')

# Limit columns
merged_df = merged_df.drop(columns=['id','name','price_clock_hour','max_num_payments'])
merged_df.head()

,started_when,account_id,to_state,group_id,is_unlocked,price_upfront,price_unlock,minimum_payment
0,2019-11-06 05:49:39.571392,2,ENABLED,1,True,950,8100,71.5
1,2019-11-25 22:20:59.150339,2,DISABLED,1,True,950,8100,71.5
2,2019-11-26 10:22:36.571392,2,ENABLED,1,True,950,8100,71.5
3,2019-11-29 11:03:32.150339,2,DISABLED,1,True,950,8100,71.5
4,2019-12-03 10:08:03.571392,2,ENABLED,1,True,950,8100,71.5


## Limit by Indicated Timestamp

In [22]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [23]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

# Adjust timestamp to end of day
timestamp_dt = timestamp_dt + timedelta(hours=23,minutes=59,seconds=59)

In [24]:
# Change data type from string to datetime object
merged_df['started_when'] = pd.to_datetime(merged_df['started_when'])

In [25]:
# Create limited dataframe based on indicated timestamp 
merged_limited_df = merged_df.loc[merged_df['started_when'] <= timestamp_dt]

### amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.

* merge groups_df w/ ast_df (may need to look at payments_df?)
* merge several: ast account_id to find product_id
* Count number of times each account has gone into disabled
* Count of times disabed TIMES(*) min payment + down payment